In [23]:
class ExcelReviewTable(BaseDBComponent):
    """Table for storing information/tracking about excel file reviews"""

    TABLE_NAME = "excel_review_table"

    def __init__(self):
        super().__init__()

    def create_excel_review_table(self):
        """Create the excel review table if it does not already exist"""
        query = f"""
            CREATE TABLE IF NOT EXISTS {self.TABLE_NAME}(
            id SERIAL PRIMARY KEY,
            filename VARCHAR(255) UNIQUE NOT NULL,   
            file_path VARCHAR(255),  -- Optional file path column
            status VARCHAR(55) NOT NULL DEFAULT 'pending',
            error_message TEXT,
            attempt_count INTEGER NOT NULL DEFAULT 0,
            last_processed_at TIMESTAMP,
            created_at TIMESTAMP NOT NULL DEFAULT NOW(),
            updated_at TIMESTAMP NOT NULL DEFAULT NOW()
            );

            -- Create index on filename for faster lookups
            CREATE INDEX IF NOT EXISTS idx_reviewed_excel_filename
            ON {self.TABLE_NAME}(filename);

            -- Create index on status for filtering by status
            CREATE INDEX IF NOT EXISTS idx_reviewed_excel_status
            ON {self.TABLE_NAME}(status);
        """
        self.execute_query(query)

In [24]:
table = ExcelReviewTable()
with table as tb:
    tb.create_excel_review_table()

In [25]:
import logging
from enum import Enum
from datetime import datetime, timedelta
from typing import Optional, Dict, List
# from app.database.base import BaseDBComponent

logger = logging.getLogger(__name__)


class MatchingStatus(str, Enum):
    """Enum for weightage matching status"""
    PENDING = 'pending'
    PROCESSING = 'processing'
    MATCHED = 'matched'
    UNMATCHED = 'unmatched'
    FAILED = 'failed'
    

class CbsDataTable(BaseDBComponent):
    """Handles database operations for CBS data synchronization"""

    TABLE_NAME = "cbs_data_table"
    SYNC_CONFIG_TABLE = "cbs_sync_config"
    DEFAULT_SYNC_INTERVAL = 24 

    def __init__(self):
        super().__init__()

    def create_table(self) -> None:
        """Create the CBS data table if it doesn't exist"""
        query = f"""
            CREATE TABLE IF NOT EXISTS {self.TABLE_NAME} (
                id SERIAL PRIMARY KEY,
                cif_id VARCHAR(255),
                cust_first_name VARCHAR(255),
                cust_middle_name VARCHAR(255),
                cust_last_name VARCHAR(255),
                fathers_name VARCHAR(255),
                grandfathers_name VARCHAR(255),
                cust_dob DATE,
                cust_cif_opn_date DATE,
                ctz_number VARCHAR(255),
                ctz_issue_date DATE,
                ctz_issued_district VARCHAR(255),
                nid_number VARCHAR(255),
                nid_issued_district VARCHAR(255),
                perm_address TEXT,
                acct_number VARCHAR(255),
                acct_name VARCHAR(255),
                acct_opn_date DATE,
                acct_status VARCHAR(50),
                frez_code VARCHAR(50),
                last_sync TIMESTAMP NOT NULL DEFAULT NOW(),
                created_at TIMESTAMP NOT NULL DEFAULT NOW(),
                updated_at TIMESTAMP NOT NULL DEFAULT NOW()
            );
            
            CREATE INDEX IF NOT EXISTS idx_cbs_cif_id ON {self.TABLE_NAME}(cif_id);
            CREATE INDEX IF NOT EXISTS idx_cbs_acct_number ON {self.TABLE_NAME}(acct_number);
            CREATE INDEX IF NOT EXISTS idx_cbs_ctz_number ON {self.TABLE_NAME}(ctz_number);
            CREATE INDEX IF NOT EXISTS idx_cbs_nid_number ON {self.TABLE_NAME}(nid_number);
        """
        try:
            self.execute_query(query)
            logger.info(f"Created/verified table {self.TABLE_NAME}")
        except Exception as e:
            logger.error(f"Error creating table: {str(e)}")
            raise

    def create_sync_config_table(self) -> None:
        """Create a table to store sync configuration and last sync time"""
        query = f"""
            CREATE TABLE IF NOT EXISTS {self.SYNC_CONFIG_TABLE} (
                id SERIAL PRIMARY KEY,
                sync_interval_hours INTEGER NOT NULL DEFAULT {self.DEFAULT_SYNC_INTERVAL},
                last_sync_time TIMESTAMP,
                sync_status VARCHAR(50) DEFAULT 'success',
                error_message TEXT,
                created_at TIMESTAMP NOT NULL DEFAULT NOW(),
                updated_at TIMESTAMP NOT NULL DEFAULT NOW()
            );
        """
        try:
            self.execute_query(query)
            logger.info(f"Created/verified table {self.SYNC_CONFIG_TABLE}")
        except Exception as e:
            logger.error(f"Error creating sync config table: {str(e)}")
            raise

    def _initialize_sync_config(self) -> None:
        """Initialize sync configuration if it doesn't exist"""
        query = f"""
            INSERT INTO {self.SYNC_CONFIG_TABLE} (sync_interval_hours, created_at, updated_at)
            SELECT {self.DEFAULT_SYNC_INTERVAL}, NOW(), NOW()
            WHERE NOT EXISTS (SELECT 1 FROM {self.SYNC_CONFIG_TABLE} LIMIT 1);
        """
        try:
            self.execute_query(query)
        except Exception as e:
            logger.error(f"Error initializing sync config: {str(e)}")
            raise

    def get_sync_config(self) -> Dict:
        """Get current sync configuration"""
        query = f"""
            SELECT * FROM {self.SYNC_CONFIG_TABLE}
            ORDER BY id DESC
            LIMIT 1;
        """
        try:
            results = self.execute_query(query)
            if results:
                return results[0]
            return {
                'sync_interval_hours': self.DEFAULT_SYNC_INTERVAL,
                'last_sync_time': None,
                'sync_status': 'none'
            }
        except Exception as e:
            logger.error(f"Error getting sync config: {str(e)}")
            raise

    def update_sync_time(self, status: str = 'success', error_message: str = None) -> None:
        """Update the last sync time and status"""
        query = f"""
            UPDATE {self.SYNC_CONFIG_TABLE}
            SET last_sync_time = NOW(),
                sync_status = %s,
                error_message = %s,
                updated_at = NOW()
            WHERE id = (SELECT id FROM {self.SYNC_CONFIG_TABLE} ORDER BY id DESC LIMIT 1);
        """
        try:
            self.execute_query(query, (status, error_message))
            logger.info(f"Updated sync time with status: {status}")
        except Exception as e:
            logger.error(f"Error updating sync time: {str(e)}")
            raise

    def is_sync_needed(self) -> bool:
        """Check if data synchronization is needed based on configured interval"""
        config = self.get_sync_config()
        last_sync_time = config.get('last_sync_time')
        sync_interval_hours = config.get('sync_interval_hours', self.DEFAULT_SYNC_INTERVAL)
        
        # If never synced or sync_status is 'failed', sync is needed
        if not last_sync_time or config.get('sync_status') == 'failed':
            return True
            
        # Calculate if enough time has passed since last sync
        current_time = datetime.now()
        time_diff = current_time - last_sync_time
        hours_passed = time_diff.total_seconds() / 3600
        
        return hours_passed >= sync_interval_hours

    def sync_cbs_data(self, cbs_data: List[Dict]) -> None:
        """Sync CBS data from view to local table"""
        try:
            self.execute_query("BEGIN;")            
            self.execute_query(f"TRUNCATE TABLE {self.TABLE_NAME}")
            
            query = f"""
                INSERT INTO {self.TABLE_NAME} (
                    cif_id, cust_first_name, cust_middle_name, cust_last_name,
                    fathers_name, grandfathers_name, cust_dob, cust_cif_opn_date,
                    ctz_number, ctz_issue_date, ctz_issued_district, nid_number,
                    nid_issued_district, perm_address, acct_number, acct_name,
                    acct_opn_date, acct_status, frez_code, last_sync, created_at, updated_at
                ) VALUES (
                    %(CIF_ID)s, %(CUST_FIRST_NAME)s, %(CUST_MIDDLE_NAME)s, %(CUST_LAST_NAME)s,
                    %(FATHERS_NAME)s, %(GRAND_FATHERS_NAME)s, %(CUST_DOB)s, %(CUST_CIF_OPN_DATE)s,
                    %(CTZ_NUMBER)s, %(CTZ_ISSUE_DATE)s, %(CTZ_ISSUED_DISTRICT)s, %(NID_NUMBER)s,
                    %(NID_ISSUEDDISTRICT)s, %(PERM_ADDDRES)s, %(ACCT_NUMBER)s, %(ACCT_NAME)s,
                    %(ACCT_OPN_DATE)s, %(ACCT_STATUS)s, %(FREZ_CODE)s, NOW(), NOW(), NOW()
                )
            """            
            batch_size = 1000
            for i in range(0, len(cbs_data), batch_size):
                batch = cbs_data[i:i + batch_size]
                logger.info(f"Processing batch {i//batch_size + 1} with {len(batch)} records")
                
                for record in batch:
                    self.execute_query(query, record)
            
            self.update_sync_time('success')
            self.execute_query("COMMIT;")            
            logger.info(f"Successfully synced {len(cbs_data)} CBS records")
        
        except Exception as e:
            # Rollback on error
            self.execute_query("ROLLBACK;")
            error_msg = str(e)
            logger.error(f"Error syncing CBS data: {error_msg}")
            
            # Record sync failure
            self.update_sync_time('failed', error_msg)
            raise

    def get_cbs_data(self, filters: Optional[Dict] = None, oracle_component=None) -> List[Dict]:
        """
        Get CBS data with optional filters
        
        If data needs to be synced and oracle_component is provided, 
        performs sync before retrieving data
        """
        try:
            # Check if sync is needed and oracle_component is available
            if oracle_component and self.is_sync_needed():
                logger.info("Data sync needed, fetching fresh data from Oracle")
                try:
                    cbs_data = oracle_component.fetch_cbs_data()
                    if cbs_data:
                        self.sync_cbs_data(cbs_data)
                    else:
                        logger.warning("No data returned from Oracle for sync")
                except Exception as e:
                    logger.error(f"Failed to sync data: {str(e)}")
                    # Continue with potentially stale data
            
            # Build and execute query with filters
            query = f"SELECT * FROM {self.TABLE_NAME}"
            params = []
            
            if filters:
                conditions = []
                for key, value in filters.items():
                    conditions.append(f"{key} = %s")
                    params.append(value)
                if conditions:
                    query += " WHERE " + " AND ".join(conditions)
            
            results = self.execute_query(query, tuple(params) if params else None)
            return results
        except Exception as e:
            logger.error(f"Error getting CBS data: {str(e)}")
            raise

    def update_sync_interval(self, hours: int) -> None:
        """Update the sync interval"""
        if hours < 1:
            raise ValueError("Sync interval must be at least 1 hour")
            
        query = f"""
            UPDATE {self.SYNC_CONFIG_TABLE}
            SET sync_interval_hours = %s,
                updated_at = NOW()
            WHERE id = (SELECT id FROM {self.SYNC_CONFIG_TABLE} ORDER BY id DESC LIMIT 1);
        """
        try:
            self.execute_query(query, (hours,))
            logger.info(f"Updated sync interval to {hours} hours")
        except Exception as e:
            logger.error(f"Error updating sync interval: {str(e)}")
            raise

    def get_sync_status(self) -> Dict:
        """Get current sync status information"""
        config = self.get_sync_config()
        last_sync_time = config.get('last_sync_time')
        sync_interval_hours = config.get('sync_interval_hours', self.DEFAULT_SYNC_INTERVAL)
        
        current_time = datetime.now()
        
        if last_sync_time:
            next_sync_time = last_sync_time + timedelta(hours=sync_interval_hours)
            time_until_next_sync = next_sync_time - current_time
            hours_until_next_sync = max(0, time_until_next_sync.total_seconds() / 3600)
        else:
            next_sync_time = None
            hours_until_next_sync = 0
            
        return {
            'last_sync_time': last_sync_time,
            'sync_interval_hours': sync_interval_hours,
            'next_sync_time': next_sync_time,
            'hours_until_next_sync': round(hours_until_next_sync, 2),
            'sync_needed': self.is_sync_needed(),
            'sync_status': config.get('sync_status', 'none'),
            'error_message': config.get('error_message')
        }

    def force_sync(self, oracle_component) -> Dict:
        """Force immediate sync regardless of interval"""
        if not oracle_component:
            raise ValueError("Oracle component is required for force sync")
            
        try:
            cbs_data = oracle_component.fetch_cbs_data()
            if cbs_data:
                self.sync_cbs_data(cbs_data)
                return {
                    'success': True,
                    'message': f"Successfully synced {len(cbs_data)} records",
                    'record_count': len(cbs_data)
                }
            else:
                return {
                    'success': False,
                    'message': "No data returned from Oracle"
                }
        except Exception as e:
            error_msg = str(e)
            return {
                'success': False,
                'message': f"Sync failed: {error_msg}"
            }



In [ ]:
import pandas as pd

cbs_data = pd.read_csv(r'C:\Users\KBL\Desktop\LetterAction_Bot1\kbl-letter-action\test\cbs_full_view.csv')
cbs_data.head()



C:\Users\KBL\AppData\Local\Temp\ipykernel_13180\2543566296.py:3: DtypeWarning: Columns (11,12,15) have mixed types. Specify dtype option on import or set low_memory=False.
  cbs_data = pd.read_csv(r'C:\Users\KBL\Desktop\LetterAction_Bot1\kbl-letter-action\test\cbs_full_view.csv')


,CIF_ID,CUST_FIRST_NAME,CUST_MIDDLE_NAME,CUST_LAST_NAME,FATHERS_NAME,GRAND_FATHERS_NAME,CUST_DOB,CUST_CIF_OPN_DATE,CTZ_NUMBER,CTZ_ISSUE_DATE,CTZ_ISSUED_DISTRICT,NID_NUMBER,NID_ISSUEDDISTRICT,PERM_ADDDRES,ACCT_NAME,ACCT_NUMBER,ACCT_OPN_DATE,ACCT_STATUS,FREZ_CODE
0,R00000001,LAXMAN,NaN,SHRESTHA,DABAL BAHADUR SHRESTHA,BHABANI SANKAR SHRESTHA,1949-06-16 00:00:00,2007-09-17,988,1973-09-17 12:00:00,KATHMANDU,NaN,NaN,KATHMANDU,LAXMAN SHRESTHA,10000000100001,2007-09-17,A,D
1,R00000002,SANTOSH,KUMAR,LAMA,PASANG LAMA,BIR BAHADUR LAMA,1955-02-21 00:00:00,2001-04-03,7950,1980-02-01 12:00:00,KATHMANDU,NaN,NaN,KATHMANDU,SANTOSH K LAMA,10000000200001,2001-04-03,A,
2,R00000003,PHURBA,WANGEL,LAMA,RINJE WANGU LAMA,SANGE LAMA,1963-05-02 00:00:00,2007-10-01,4345,1989-07-12 12:00:00,SUNSARI,NaN,NaN,SUNSARI,PHURBA WANGEL LAMA,10000000300001,2007-10-01,A,
3,R00000006,MAHESH,PRASAD,BHATTARAI,SANAT PRASAD BHATTARAI,MAYA NATH BHATTARAI,1960-04-15 00:00:00,2004-05-20,8140/18294,1978-06-23 12:00:00,KOSHI,NaN,NaN,KOSHI,MAHESH PRASAD BHATTARAI,10000000600001,2004-05-20,A,
4,R00000008,SEETA,NaN,GURUNG,KRISHNA BAHADUR GHALE,MAN BAHADUR GHALE,1948-10-03 00:00:00,2001-04-03,2089/1998,2007-02-12 12:00:00,KATHMANDU,NaN,NaN,GANDAKI,SEETA GURUNG,10000000800001,2001-04-03,D,D


In [27]:
df = cbs_data.where(pd.notna(cbs_data), None)

# Convert the DataFrame to a list of dictionaries
list_of_dicts = df.to_dict(orient='records')

list_of_dicts[:5]

[{'CIF_ID': 'R00000001',
  'CUST_FIRST_NAME': 'LAXMAN',
  'CUST_MIDDLE_NAME': None,
  'CUST_LAST_NAME': 'SHRESTHA',
  'FATHERS_NAME': 'DABAL BAHADUR SHRESTHA',
  'GRAND_FATHERS_NAME': 'BHABANI SANKAR SHRESTHA',
  'CUST_DOB': '1949-06-16 00:00:00',
  'CUST_CIF_OPN_DATE': '2007-09-17',
  'CTZ_NUMBER': '988',
  'CTZ_ISSUE_DATE': '1973-09-17 12:00:00',
  'CTZ_ISSUED_DISTRICT': 'KATHMANDU',
  'NID_NUMBER': None,
  'NID_ISSUEDDISTRICT': None,
  'PERM_ADDDRES': 'KATHMANDU',
  'ACCT_NAME': 'LAXMAN SHRESTHA',
  'ACCT_NUMBER': 10000000100001,
  'ACCT_OPN_DATE': '2007-09-17',
  'ACCT_STATUS': 'A',
  'FREZ_CODE': 'D'},
 {'CIF_ID': 'R00000002',
  'CUST_FIRST_NAME': 'SANTOSH',
  'CUST_MIDDLE_NAME': 'KUMAR',
  'CUST_LAST_NAME': 'LAMA',
  'FATHERS_NAME': 'PASANG LAMA',
  'GRAND_FATHERS_NAME': 'BIR BAHADUR LAMA',
  'CUST_DOB': '1955-02-21 00:00:00',
  'CUST_CIF_OPN_DATE': '2001-04-03',
  'CTZ_NUMBER': '7950',
  'CTZ_ISSUE_DATE': '1980-02-01 12:00:00',
  'CTZ_ISSUED_DISTRICT': 'KATHMANDU',
  'NID_NUMBER

In [28]:
cbs_table = CbsDataTable()
with cbs_table as db:
    db.create_table()
    db.sync_cbs_data(list_of_dicts)